In [72]:
import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
rest_reviews=pd.read_pickle('/Volumes/DrewM_External/DataScience/Incubator_Challenge/Yelp_data/yelp_dataset/restaurant_reviews')

In [3]:
business=pd.read_feather('/Volumes/DrewM_External/DataScience/Incubator_Challenge/Yelp_data/yelp_dataset/business')

In [4]:
restaurants=business[business['categories'].str.contains("Restaurants",na=False)]

In [5]:
unq_rest_ids=restaurants['business_id'].unique()

In [6]:
user_id_count=rest_reviews.user_id.value_counts().index.tolist()

In [21]:
bus_ids_for_user=rest_reviews[rest_reviews['user_id']==user_id_count[0]].business_id.unique()

In [ ]:
all_reviews_for_bus=rest_reviews[rest_reviews['business_id']==bus_ids_for_user[0]]

In [27]:
user_reviews_for_bus=all_reviews_for_bus[all_reviews_for_bus['user_id']==user_id_count[0]]

In [31]:
catagory=np.ceil(np.mean(user_reviews_for_bus.stars))

In [33]:
other_reviews_for_bus=all_reviews_for_bus[all_reviews_for_bus['user_id']!=user_id_count[0]]

In [39]:
text=' '.join(other_reviews_for_bus.text)

In [43]:
test_bus=bus_ids_for_user[0:50]

In [44]:
train_bus=bus_ids_for_user[50:]

In [85]:
count = 0
cat=[]
data=[]
mean_star=[]
for bus_id in bus_ids_for_user:
    all_reviews_for_bus=rest_reviews[rest_reviews['business_id']==bus_id]
    user_reviews_for_bus=all_reviews_for_bus[all_reviews_for_bus['user_id']==user_id_count[0]]
    catagory=np.ceil(np.mean(user_reviews_for_bus.stars))
    other_reviews_for_bus=all_reviews_for_bus[all_reviews_for_bus['user_id']!=user_id_count[0]]
    text=' '.join(other_reviews_for_bus.text)
    cat.append(catagory)
    data.append(text)
    mean_star.append(np.round(np.mean(other_reviews_for_bus.stars)))
    count += 1

In [56]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, min_df=5, stop_words='english')
X_train = vectorizer.fit_transform(data[50:])
X_test = vectorizer.transform(data[0:50])

In [58]:
Y_train = cat[50:]
Y_test = cat[0:50]

In [65]:
feature_names = vectorizer.get_feature_names()
clf=BernoulliNB(alpha=.01)
clf.fit(X_train, Y_train)

BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=True)

In [66]:
pred = clf.predict(X_test)

In [70]:
score = metrics.accuracy_score(Y_test, pred)

In [71]:
score

0.36

In [78]:
np.mean(np.abs(Y_test-pred))

0.92

In [84]:
np.round(np.mean(other_reviews_for_bus.stars))

2.0

In [88]:
metrics.accuracy_score(Y_test, mean_star[0:50])

0.4

In [90]:
np.mean(np.abs(Y_test-np.array(mean_star[0:50])))

0.64